In [1]:
pip install ipywidgets

Looking in indexes: http://repo.myhuaweicloud.com/repository/pypi/simple
     |████████████████████████████████| 139 kB 21.5 MB/s eta 0:00:01
     |████████████████████████████████| 2.2 MB 40.9 MB/s eta 0:00:01
     |████████████████████████████████| 216 kB 15.4 MB/s eta 0:00:01
You should consider upgrading via the '/home/ma-user/anaconda3/envs/PyTorch-1.10.2/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [2]:
# ## 1. 导入必要的库
import numpy as np
import torch
import torch.nn.functional as F
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset # 生成时非必需，但为原始notebook内容
import torch.optim as optim # 生成时非必需
import random
import time

In [3]:
# ## 2. 加载数据和映射关系
# 确保 tang.npz 文件在同一目录下，或提供正确的路径
try:
    tang_file = np.load("tang.npz", allow_pickle=True)
    word2ix = tang_file['word2ix'].item()
    idx2word = tang_file['ix2word'].item()
    data = tang_file['data'] # 为上下文加载，在下面的生成函数中不直接使用
    print("数据和映射关系加载完成。")
    print(f"词汇表大小: {len(word2ix)}")
except FileNotFoundError:
    print("错误: tang.npz 文件未找到。请确保它在正确的路径下。")
    # 使用虚拟值初始化，以允许脚本进一步运行，但生成会失败。
    word2ix = {"<START>": 0, "<EOP>": 1, "，": 2, "。": 3, "山":4, "水":5, "重":6, "庆":7, "大":8, "好":9, "河":10, "<unk>": 11}
    idx2word = {v: k for k, v in word2ix.items()}
    data = []

数据和映射关系加载完成。
词汇表大小: 8293


In [4]:
# ## 3. 定义模型 (`PoemNet`)
# 这应该与训练时使用的定义相同。
class PoemNet(nn.Module):
    def __init__(self, vocab_size_param, embedding_dim_param, hidden_dim_param):
        super(PoemNet, self).__init__()
        self.embeddings = nn.Embedding(vocab_size_param, embedding_dim_param)
        self.lstm = nn.LSTM(embedding_dim_param, hidden_dim_param, num_layers=2, batch_first=True, dropout=0.5)
        self.fc = nn.Sequential(
            nn.Linear(hidden_dim_param, 2048),
            nn.ReLU(),
            nn.Dropout(0.25),
            nn.Linear(2048, 4096),
            nn.Dropout(0.2),
            nn.ReLU(),
            nn.Linear(4096, vocab_size_param)
        )

    def forward(self, input_seq, hidden=None):
        embedded = self.embeddings(input_seq)
        lstm_out, hidden = self.lstm(embedded, hidden)
        output = self.fc(lstm_out)
        return output, hidden

In [7]:
# ## 4. 加载已训练模型
# Determine the device to use (CUDA GPU if available, otherwise CPU)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"使用的设备: {device}") # Display the device being used

VOCAB_SIZE = len(word2ix) 
EMBEDDING_DIM = 200
HIDDEN_DIM = 1024

my_net = None # Initialize my_net to None

try:
    # User guarantees that "model.h5" is a complete model file, so load it directly.
    print("尝试加载完整模型 model.h5...")
    # Load the entire model from the specified file.
    # map_location=device ensures the model is loaded onto the correct device from the start.
    my_net = torch.load("model.h5", map_location=device)

    my_net = my_net.to(device)
    
    if my_net: # Check if the model was successfully loaded and assigned to my_net
        my_net.eval() # Set the model to evaluation mode. This disables layers like Dropout.
        print("模型 model.h5 加载成功。")
    else:
        # This case should ideally be caught by an exception if torch.load fails (e.g., returns None or a non-model object).
        # Kept as an additional safeguard.
        print("错误：模型未能正确加载。torch.load 可能返回了 None 或非预期对象。")

except FileNotFoundError:
    # Handle the case where the model file does not exist at the specified path.
    print("错误: model.h5 文件未找到。请确保它在正确的路径下。")
    my_net = None # Ensure my_net remains None if loading fails
except Exception as e:
    # Catch other potential errors during model loading, such as a corrupted file or an invalid PyTorch model file.
    print(f"加载模型 model.h5 时发生一般性错误: {e}")
    print("请确保 model.h5 是一个有效的 PyTorch 完整模型文件。")
    my_net = None # Ensure my_net remains None if loading fails


使用的设备: cuda:0
尝试加载完整模型 model.h5...
模型 model.h5 加载成功。


In [8]:
# ## 5. generate_poem 方法 (根据 docx 结构补全)
def generate_poem(my_words, num_chars_per_line): # 移除了 N_GRAM_SIZE 参数
    '''
    根据前文my_words生成一首固定四句话的诗。
    my_words: 提示词字符串。
    num_chars_per_line: 每行诗的字数 (5或7)。
    N_GRAM_SIZE 将硬编码为 2 (二元组检查)。
    '''
    N_GRAM_SIZE = 2 # 硬编码 N_GRAM_SIZE 为 2

    if my_net is None or not word2ix or not idx2word:
        return "错误：模型或数据映射未正确加载。"
    if num_chars_per_line not in [5, 7]:
        return "错误：每行字数必须是5或7。"

    def is_chinese_char(char_unicode): # 判断是否为中文字符的辅助函数
        if char_unicode is None: return False
        if '\u4e00' <= char_unicode <= '\u9fff':
            return True
        return False
        
    def __generate_next(idx, hidden=None): 
        """
        根据输入索引和隐藏状态输出下一个预测词的logits和新的隐藏状态。
        """
        input_tensor = torch.tensor([[idx]], dtype=torch.long).to(device)
        output_logits, new_hidden = my_net(input_tensor, hidden)
        return output_logits, new_hidden

    start_token_idx = word2ix.get("<START>") 
    if start_token_idx is None: return "错误: <START> 不在词典中。"
    
    current_logits, hidden = __generate_next(start_token_idx) 

    poem_lines = [] 
    current_line_chars = [] # 存储当前正在构建的诗句中的汉字
    
    # 用于n-gram检查的集合，存储当前行已经出现过的n-gram字符串
    current_line_ngrams = set() 

    unk_token_idx = word2ix.get('<unk>') 
    
    default_fallback_char_str = "之" 
    fallback_char_idx = word2ix.get(default_fallback_char_str)
    actual_fallback_char_for_display = default_fallback_char_str

    if fallback_char_idx is None: 
        if unk_token_idx is not None:
            fallback_char_idx = unk_token_idx
            actual_fallback_char_for_display = idx2word.get(unk_token_idx, "<unk>")
            print(f"警告: 回退字符 '之' 不在词典中，将使用 '{actual_fallback_char_for_display}'。")
        else: 
            found_alternative_fallback = False
            for idx_val_iter, char_val_iter in idx2word.items():
                if is_chinese_char(char_val_iter):
                    fallback_char_idx = idx_val_iter
                    actual_fallback_char_for_display = char_val_iter
                    print(f"警告: 回退字符 '之' 和 '<unk>' 均不在词典中，将使用词典中的 '{actual_fallback_char_for_display}'。")
                    found_alternative_fallback = True
                    break
            if not found_alternative_fallback:
                 return "错误：词汇表中没有可用的中文回退字符 (例如 '之' 或 '<unk>')。"

    # 1. 处理提示词 (my_words)
    processed_prompt_chars_for_line = [] 
    temp_current_logits_after_prompt = current_logits 
    temp_hidden_after_prompt = hidden 

    if my_words: 
        for char_in_prompt in my_words:
            if len(poem_lines) == 4: 
                break
            
            char_idx_for_feed = word2ix.get(char_in_prompt)
            if char_idx_for_feed is None:
                if unk_token_idx is not None:
                    print(f"警告：提示词中的字 '{char_in_prompt}' 不在词典中，将使用'<unk>'影响隐藏状态。")
                    char_idx_for_feed = unk_token_idx
                else: 
                    print(f"警告：提示词中的字 '{char_in_prompt}' 未知且无'<unk>'，跳过此字对隐藏状态的影响。")
                    continue 

            temp_current_logits_after_prompt, temp_hidden_after_prompt = __generate_next(char_idx_for_feed, temp_hidden_after_prompt)

            if is_chinese_char(char_in_prompt):
                processed_prompt_chars_for_line.append(char_in_prompt)
                # 更新提示词部分的n-gram (如果N_GRAM_SIZE > 1)
                if N_GRAM_SIZE > 1 and len(processed_prompt_chars_for_line) >= N_GRAM_SIZE:
                    current_line_ngrams.add("".join(processed_prompt_chars_for_line[-(N_GRAM_SIZE):]))

                if len(processed_prompt_chars_for_line) == num_chars_per_line:
                    poem_lines.append("".join(processed_prompt_chars_for_line))
                    processed_prompt_chars_for_line = []
                    current_line_ngrams.clear() # 新的一行，清空n-gram记录
        
        current_logits = temp_current_logits_after_prompt
        hidden = temp_hidden_after_prompt
        
        current_line_chars.extend(processed_prompt_chars_for_line)


    # 2. 生成诗歌的其余部分 (目标是总共4行)
    while len(poem_lines) < 4:
        while len(current_line_chars) < num_chars_per_line:
            output_logits_squeezed = current_logits.squeeze()
            top_k_probs, top_k_indices = torch.topk(output_logits_squeezed, k=10) 
            
            chosen_char_for_line = None
            chosen_idx_for_next_step = -1
            
            # 尝试从top_k中选择一个不构成重复n-gram的字符
            for i_k in range(top_k_indices.size(0)):
                potential_idx = top_k_indices[i_k].item()
                potential_char = idx2word.get(potential_idx)

                if potential_char and potential_char != "<EOP>" and is_chinese_char(potential_char):
                    is_ngram_repeated = False
                    if N_GRAM_SIZE > 1 and len(current_line_chars) >= N_GRAM_SIZE - 1:
                        # 构建测试的n-gram
                        # 前 N_GRAM_SIZE - 1 个字符来自 current_line_chars 的末尾
                        # 最后一个字符是 potential_char
                        prefix_for_ngram = "".join(current_line_chars[-(N_GRAM_SIZE - 1):])
                        test_ngram = prefix_for_ngram + potential_char
                        if test_ngram in current_line_ngrams:
                            is_ngram_repeated = True
                    
                    if not is_ngram_repeated:
                        chosen_char_for_line = potential_char
                        chosen_idx_for_next_step = potential_idx
                        break # 找到了合适的非重复字符
            
            if chosen_char_for_line is None: # 如果所有top_k候选都导致重复或不合适
                # 退回到选择第一个有效的中文字符 (即使它可能导致重复)
                # 或者使用全局回退字符
                # 为了保证生成，我们这里选择第一个有效中文字符，或者全局回退
                first_valid_candidate_char = None
                first_valid_candidate_idx = -1
                for i_k_fallback in range(top_k_indices.size(0)):
                    potential_idx_fb = top_k_indices[i_k_fallback].item()
                    potential_char_fb = idx2word.get(potential_idx_fb)
                    if potential_char_fb and potential_char_fb != "<EOP>" and is_chinese_char(potential_char_fb):
                        first_valid_candidate_char = potential_char_fb
                        first_valid_candidate_idx = potential_idx_fb
                        break
                
                if first_valid_candidate_char:
                    chosen_char_for_line = first_valid_candidate_char
                    chosen_idx_for_next_step = first_valid_candidate_idx
                    # print(f"提示: N-gram检查后无优选，选择top-k首个有效字符'{chosen_char_for_line}'")
                else: # 连top-k里都没有有效中文字符了
                    chosen_char_for_line = actual_fallback_char_for_display
                    chosen_idx_for_next_step = fallback_char_idx
                    # print(f"提示: N-gram检查后无优选且top-k无效，使用全局回退字符 '{chosen_char_for_line}'")


            current_line_chars.append(chosen_char_for_line)
            # 更新当前行的n-gram记录
            if N_GRAM_SIZE > 1 and len(current_line_chars) >= N_GRAM_SIZE:
                current_line_ngrams.add("".join(current_line_chars[-(N_GRAM_SIZE):]))

            current_logits, hidden = __generate_next(chosen_idx_for_next_step, hidden)
        
        poem_lines.append("".join(current_line_chars))
        current_line_chars = [] 
        current_line_ngrams.clear() # 新的一行，清空n-gram记录
        
    # 3. 格式化输出
    if not poem_lines: 
        return "（未生成诗歌）"

    formatted_poem_str = "" 
    for i, line_content in enumerate(poem_lines): 
        formatted_poem_str += line_content 
        if i == len(poem_lines) - 1: 
            formatted_poem_str += "。" 
        else: 
            formatted_poem_str += "，\n" 
            
    return formatted_poem_str

In [10]:
# 测试 generate_poem
if my_net:
    print("\n--- 测试 generate_poem (五言) ---")
    generated_text_5_ngram = generate_poem("重庆", num_chars_per_line=5)
    print(f"generate_poem(\"重庆\", 5) 输出:\n{generated_text_5_ngram}\n")
    
    print("\n--- 测试 generate_poem (五言) ---")
    generated_text_5_ngram = generate_poem("", 5)
    print(f"generate_poem(\"无提示词\", 5) 输出:\n{generated_text_5_ngram}\n")
    
    print("\n--- 测试 generate_poem (五言) ---")
    generated_text_full_prompt_5_ngram = generate_poem("春眠不觉晓", 5)
    print(f"generate_poem(\"春眠不觉晓\", 5) 输出:\n{generated_text_full_prompt_5_ngram}\n")

    print("\n--- 测试 generate_poem (七言) ---")
    generated_text_7_ngram = generate_poem("青山隐隐", num_chars_per_line=7)
    print(f"generate_poem(\"青山隐隐\", 7) 输出:\n{generated_text_7_ngram}\n")

    print("\n--- 测试 generate_poem (七言) ---")
    generated_text_7_ngram = generate_poem("我爱重庆大学", 7)
    print(f"generate_poem(\"我爱重庆大学\", 7) 输出:\n{generated_text_7_ngram}\n")
    
    print("\n--- 测试 generate_poem (七言) ---")
    generated_text_7_ngram = generate_poem("自然语言处理", 7)
    print(f"generate_poem(\"自然语言处理\", 7) 输出:\n{generated_text_7_ngram}\n")

    print("\n--- 测试 generate_poem (提示词超出一行，五言) ---") 
    generated_text_overflow_prompt_5_ngram3 = generate_poem("白日依山尽黄河", 5)
    print(f"generate_poem(\"白日依山尽黄河\", 5) 输出:\n{generated_text_overflow_prompt_5_ngram3}\n")
    
    print("\n--- 测试 generate_poem (提示词超出一行，七言) ---") 
    generated_text_overflow_prompt_7_ngram3 = generate_poem("故人西辞黄鹤楼烟花", 7)
    print(f"generate_poem(\"故人西辞黄鹤楼烟花\", 7) 输出:\n{generated_text_overflow_prompt_7_ngram3}\n")

else:
    print("generate_poem: 模型未加载，无法测试。")


--- 测试 generate_poem (五言) ---
generate_poem("重庆", 5) 输出:
重庆三千里，
上游仙客处，
寻常到处寻，
常久处栽花。


--- 测试 generate_poem (五言) ---
generate_poem("无提示词", 5) 输出:
一片寒泉一，
片云鬟月映，
明月中天月，
明月光明灭。


--- 测试 generate_poem (五言) ---
generate_poem("春眠不觉晓", 5) 输出:
春眠不觉晓，
光迟日迟迟，
迟迟日迟回，
日迟迟日暮。


--- 测试 generate_poem (七言) ---
generate_poem("青山隐隐", 7) 输出:
青山隐隐沦天地，
宽深处处寻常在，
处寻常处处无人，
到处无人到世间。


--- 测试 generate_poem (七言) ---
generate_poem("我爱重庆大学", 7) 输出:
我爱重庆大学名，
张李李杜公子诗，
人不可论高臥玉，
堂中夜月明月光。


--- 测试 generate_poem (七言) ---
generate_poem("自然语言处理", 7) 输出:
自然语言处理衰，
荣久不知衰朽事，
牵愁处处无人识，
世情难尽处深处。


--- 测试 generate_poem (提示词超出一行，五言) ---
generate_poem("白日依山尽黄河", 5) 输出:
白日依山尽，
黄河水水流，
无穷处处无，
人到处寻常。


--- 测试 generate_poem (提示词超出一行，七言) ---
generate_poem("故人西辞黄鹤楼烟花", 7) 输出:
故人西辞黄鹤楼，
烟花之东流水上，
天涯去不回头去，
不回头白头时时。



In [11]:
# ## 6. acrostic_poetry 方法 (根据 docx 结构补全)
# acrostic_poetry 的 n-gram 优化逻辑与 generate_poem 类似。
def acrostic_poetry(my_words, num_chars_per_line): 
    if my_net is None or not word2ix or not idx2word:
        print("错误：模型或数据映射未正确加载。")
        return
    if num_chars_per_line not in [5, 7]:
        print("错误：每行字数必须是5或7。")
        return

    def is_chinese_char(char): # 内部辅助函数：判断是否为中文字符
        if char is None: return False
        return '\u4e00' <= char <= '\u9fff'

    def __generate_next_acrostic(idx, hidden=None): # 重命名以避免在脚本环境中与其他__generate_next冲突
        input_tensor = torch.tensor([[idx]], dtype=torch.long).to(device)
        output_logits, new_hidden = my_net(input_tensor, hidden)
        return output_logits, new_hidden

    def __generate_line_acrostic(word_head, hidden_state, current_num_chars_per_line): # 生成藏头诗的一行
        sentence_chars_for_line = [word_head] # 诗行以藏头字开始
        current_line_ngrams_ac = set() # 用于当前藏头诗行的n-gram记录
        
        unk_token_idx_ac = word2ix.get("<unk>") # 未知标记索引
        current_char_idx_ac = word2ix.get(word_head) # 藏头字索引

        if current_char_idx_ac is None: 
            print(f"警告：藏头字 '{word_head}' 不在词典中。将使用 '<unk>' (如果存在) 更新隐藏状态。")
            if unk_token_idx_ac is not None:
                current_char_idx_ac = unk_token_idx_ac
            else:
                print(f"错误：藏头字 '{word_head}' 未知且词典中无'<unk>'。此行可能不准确或不完整。")
                default_fallback_char_str_ac = "之"
                fallback_char_idx_ac = word2ix.get(default_fallback_char_str_ac, unk_token_idx_ac) 
                if fallback_char_idx_ac is None: 
                    found_fallback_ac = False
                    for idx_val_ac, char_val_iter_ac in idx2word.items():
                        if is_chinese_char(char_val_iter_ac):
                            fallback_char_idx_ac = idx_val_ac
                            found_fallback_ac = True
                            break
                    if not found_fallback_ac:
                        print("错误：无法为藏头诗内容生成找到回退字符。")
                        return sentence_chars_for_line, hidden_state 
                current_char_idx_ac = fallback_char_idx_ac 

        temp_logits, temp_hidden = __generate_next_acrostic(current_char_idx_ac, hidden_state) 
        
        num_additional_chars_to_generate = current_num_chars_per_line - 1 
        
        default_fallback_content_char_str = "之"
        actual_fallback_content_char_display = default_fallback_content_char_str
        fallback_content_char_idx = word2ix.get(default_fallback_content_char_str)

        if fallback_content_char_idx is None:
            if unk_token_idx_ac is not None:
                fallback_content_char_idx = unk_token_idx_ac
                actual_fallback_content_char_display = idx2word.get(unk_token_idx_ac, "<unk>")
            else: 
                found_alt_fallback_ac = False
                for idx_val_iter_ac, char_val_iter_ac in idx2word.items():
                    if is_chinese_char(char_val_iter_ac):
                        fallback_content_char_idx = idx_val_iter_ac
                        actual_fallback_content_char_display = char_val_iter_ac
                        found_alt_fallback_ac = True
                        break
                if not found_alt_fallback_ac:
                    print("错误: 藏头诗内容生成部分无法找到回退字符!")
                    return sentence_chars_for_line, temp_hidden 

        current_logits_for_loop = temp_logits 
        N_GRAM_SIZE_ACROSTIC = 2 # 为藏头诗也定义一个n-gram大小，可以设为参数

        for _ in range(num_additional_chars_to_generate): 
            top_k_probs_ac, top_k_indices_ac = torch.topk(current_logits_for_loop.squeeze(), k=5) 
            selected_char_candidate = None 
            selected_idx_candidate = -1 

            for i_k_ac in range(top_k_indices_ac.size(0)):
                potential_idx_ac = top_k_indices_ac[i_k_ac].item()
                potential_char_ac = idx2word.get(potential_idx_ac)
                if potential_char_ac and is_chinese_char(potential_char_ac):
                    is_ngram_repeated_ac = False
                    if N_GRAM_SIZE_ACROSTIC > 1 and len(sentence_chars_for_line) >= N_GRAM_SIZE_ACROSTIC -1 : 
                        prefix_ac = "".join(sentence_chars_for_line[-(N_GRAM_SIZE_ACROSTIC -1):])
                        test_ngram_ac = prefix_ac + potential_char_ac
                        if test_ngram_ac in current_line_ngrams_ac:
                            is_ngram_repeated_ac = True
                    
                    if not is_ngram_repeated_ac:
                        selected_char_candidate = potential_char_ac
                        selected_idx_candidate = potential_idx_ac
                        break
            
            char_to_append_to_line_ac: str 
            idx_for_next_feed_ac: int

            if selected_char_candidate: 
                char_to_append_to_line_ac = selected_char_candidate
                idx_for_next_feed_ac = selected_idx_candidate
            else: 
                char_to_append_to_line_ac = actual_fallback_content_char_display
                idx_for_next_feed_ac = fallback_content_char_idx
            
            sentence_chars_for_line.append(char_to_append_to_line_ac)
            if N_GRAM_SIZE_ACROSTIC > 1 and len(sentence_chars_for_line) >= N_GRAM_SIZE_ACROSTIC:
                current_line_ngrams_ac.add("".join(sentence_chars_for_line[-N_GRAM_SIZE_ACROSTIC:]))
            
            if _ < num_additional_chars_to_generate -1: 
                 current_logits_for_loop, temp_hidden = __generate_next_acrostic(idx_for_next_feed_ac, temp_hidden)

        return sentence_chars_for_line, temp_hidden 


    start_token_idx_ac = word2ix.get("<START>") 
    if start_token_idx_ac is None:
        print("错误: <START> 不在词典中 (acrostic_poetry)。")
        return

    _, overall_hidden_state = __generate_next_acrostic(start_token_idx_ac)
    
    result_lines_strings = [] 
    head_char_list = list(my_words) 

    for head_char_for_line_iter in head_char_list: 
      line_char_list_ac, overall_hidden_state = __generate_line_acrostic(
          head_char_for_line_iter, 
          overall_hidden_state, 
          num_chars_per_line
      )
      result_lines_strings.append("".join(line_char_list_ac)) 
    
    output_string_parts = []
    for i, line_text_ac in enumerate(result_lines_strings): 
        if i == len(result_lines_strings) - 1: 
            output_string_parts.append(line_text_ac + "。") 
        else: 
            output_string_parts.append(line_text_ac + "，") 

    print("\n".join(output_string_parts)) 

In [13]:
# 测试 acrostic_poetry
if my_net:
    print("\n--- 测试 acrostic_poetry (五言) ---")
    acrostic_poetry("大好河山", 5) 

    print("\n--- 测试 acrostic_poetry (五言) ---")
    acrostic_poetry("我想吃面条", 5) 
    
    print("\n--- 测试 acrostic_poetry (七言) ---")
    acrostic_poetry("春花十里", 7) 
    
    print("\n--- 测试 acrostic_poetry (七言) ---")
    acrostic_poetry("好想睡大觉", 7) 
else:
    print("acrostic_poetry: 模型未加载，无法测试。")


--- 测试 acrostic_poetry (五言) ---
大道无心处，
好不知之心，
河水之中央，
山之水堋堋。

--- 测试 acrostic_poetry (五言) ---
我家山水上，
想无穷处处，
吃然不知何，
面上天涯去，
条似雪中人。

--- 测试 acrostic_poetry (七言) ---
春风吹柳絮如丝，
花处处春深绿野，
十年时处处寻春，
里处处寻常少处。

--- 测试 acrostic_poetry (七言) ---
好鸟不知归去去，
想无人到处寻常，
睡不得行人说得，
大师人不识人间，
觉处处寻常说处。
